#Dataset Generation

# Generation of supply data

In [ ]:
import pandas as pd
import random

random.seed(42)

num_supply_nodes = 20
resources = ['food', 'water', 'medicine', 'cloth']


supply_nodes = [f"S{i+1}" for i in range(num_supply_nodes)]
supply_data = []

for supply_node in supply_nodes:
    for resource in resources:
        capacity = random.randint(900, 1400)
        supply_data.append({
            "supply_node": supply_node,
            "resource": resource,
            "capacity": capacity
        })

supply_df = pd.DataFrame(supply_data)
supply_df.to_csv('supply_data.csv', index=False)

print("`supply_data.csv` generated.")


`supply_data.csv` generated.


# Generation of demand data

In [ ]:
random.seed(42)

num_demand_nodes = 57
resources = ['food', 'water', 'medicine', 'cloth']


demand_nodes = [f"D{i+1}" for i in range(num_demand_nodes)]
demand_data = {resource: [] for resource in resources}
total_demand = {resource: 0 for resource in resources}


for demand_node in demand_nodes:
    urgency = random.choice([0, 1])
    for resource in resources:
        demand = random.randint(200, 500)
        demand_data[resource].append({
            "demand_node": demand_node,
            "demand": demand,
            "urgency": urgency
        })
        total_demand[resource] += demand

flat_demand_data = []
for resource in resources:
    for item in demand_data[resource]:
        flat_demand_data.append({
            "demand_node": item["demand_node"],
            "resource": resource,
            "demand": item["demand"],
            "urgency": item["urgency"]
        })

demand_df = pd.DataFrame(flat_demand_data)
demand_df.to_csv('demand_data.csv', index=False)

print("`demand_data.csv` generated.")


`demand_data.csv` generated.


# generation of distance data

In [ ]:
import pandas as pd
import random

random.seed(42)

num_supply_nodes = 20
num_demand_nodes = 57
items = ['food', 'water', 'medicine', 'cloth']

supply_nodes = [f"S{i+1}" for i in range(num_supply_nodes)]
demand_nodes = [f"D{i+1}" for i in range(num_demand_nodes)]
distance_data = []

for supply_node in supply_nodes:
    for demand_node in demand_nodes:
        distance = round(random.uniform(3, 20), 1)
        costs = {item: round(random.uniform(4, 15), 1) for item in items}
        distance_data.append({
            "supply_node": supply_node,
            "demand_node": demand_node,
            "distance": distance,
            **costs
        })

distance_df = pd.DataFrame(distance_data)
distance_df.to_csv('distance_data.csv', index=False)

print("`distance_data.csv` generated.")

`distance_data.csv` generated.


# Checking Supply and demands

In [ ]:
demand_df = pd.read_csv('demand_data.csv')

total_demand_by_item = demand_df.groupby('resource')['demand'].sum()
print("Total Demand by Item:\n", total_demand_by_item)

supply_df = pd.read_csv('supply_data.csv')

total_supply_by_item = supply_df.groupby('resource')['capacity'].sum()
print("\nTotal Supply Capacity by Item:\n", total_supply_by_item)

Total Demand by Item:
 resource
cloth       20240
food        19878
medicine    19526
water       19130
Name: demand, dtype: int64

Total Supply Capacity by Item:
 resource
cloth       22700
food        23799
medicine    22203
water       21582
Name: capacity, dtype: int64


In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package

#Dataset loading

In [ ]:
import pandas as pd
import pyomo.environ as pe

distance_data = pd.read_csv('distance_data.csv')
demand_data = pd.read_csv('demand_data.csv')
supply_data = pd.read_csv('supply_data.csv')

In [ ]:
distance_data

,supply_node,demand_node,distance,food,water,medicine,cloth
0,S1,D1,13.9,4.3,7.0,6.5,12.1
1,S1,D2,14.5,13.8,5.0,8.6,4.3
2,S1,D3,6.7,9.6,4.3,6.2,11.1
3,S1,D4,12.3,6.4,10.5,12.9,4.1
4,S1,D5,16.7,11.7,7.7,5.7,14.5
...,...,...,...,...,...,...,...
1135,S20,D53,18.8,9.0,8.3,5.5,11.9
1136,S20,D54,13.7,14.7,6.2,5.4,11.6
1137,S20,D55,4.0,5.5,13.4,9.0,5.9
1138,S20,D56,8.8,13.8,10.6,12.7,11.0


In [ ]:
demand_data

,demand_node,resource,demand,urgency
0,D1,food,212,0
1,D2,food,252,0
2,D3,food,215,0
3,D4,food,487,0
4,D5,food,429,0
...,...,...,...,...
223,D53,cloth,229,1
224,D54,cloth,320,0
225,D55,cloth,496,1
226,D56,cloth,489,0


In [ ]:
supply_data

,supply_node,resource,capacity
0,S1,food,1227
1,S1,water,957
2,S1,medicine,912
3,S1,cloth,1279
4,S2,food,1040
...,...,...,...
75,S19,cloth,1353
76,S20,food,1341
77,S20,water,1085
78,S20,medicine,1195


In [ ]:


supply_nodes = supply_data['supply_node'].unique().tolist()
demand_nodes = demand_data['demand_node'].unique().tolist()
resources = supply_data['resource'].unique().tolist()

supply_dict = supply_data.pivot(index='supply_node', columns='resource', values='capacity').to_dict(orient='index')
demand_dict = demand_data.pivot(index='demand_node', columns='resource', values='demand').to_dict(orient='index')

urgency = demand_data.set_index('demand_node')['urgency'].to_dict()
urgent_nodes = [node for node, is_urgent in urgency.items() if is_urgent == 1]
non_urgent_nodes = [node for node, is_urgent in urgency.items() if is_urgent == 0]

distance_dict = {
    (row['supply_node'], row['demand_node']): row['distance']
    for _, row in distance_data.iterrows()
}
cost_dict = {
    (row['supply_node'], row['demand_node'], resource): row[f'{resource}']
    for _, row in distance_data.iterrows()
    for resource in resources
}


#Step 1 : Satisfying the Urgent demands

In [ ]:
!pip install pyomo
!apt-get install -y -qq coinor-cbc  # To install CBC as a backup solver
!apt-get install -y -qq coinor-ipopt


Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 123833 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [ ]:
# Step 1: Minimize distance for urgent nodes
model1 = pe.ConcreteModel()

# Variables
model1.x = pe.Var(supply_nodes, demand_nodes, resources, domain=pe.NonNegativeReals)


# Objective: Minimize total distance for urgent nodes
def obj_rule1(model):
    return sum(
        model1.x[i,j,r]*distance_dict[(i, j)]
        for i in supply_nodes
        for j in urgent_nodes
        for r in resources
        if (i, j) in distance_dict
    )
model1.obj = pe.Objective(rule=obj_rule1, sense=pe.minimize)

# Supply constraints
def supply_constraint1(model, i, r):
    return sum(model.x[i, j, r] for j in urgent_nodes if (i, j) in distance_dict) <= supply_dict[i][r]
model1.supply_constraint = pe.Constraint(supply_nodes, resources, rule=supply_constraint1)

# Demand constraints
def demand_constraint1(model, j, r):
    return sum(model.x[i, j, r] for i in supply_nodes if (i, j) in distance_dict) >= demand_dict[j][r]
model1.demand_constraint = pe.Constraint(urgent_nodes, resources, rule=demand_constraint1)

# Solve
solver = pe.SolverFactory('glpk')
solver.solve(model1)

# Record results from Step 1
print("Step 1: Urgent Nodes Results")
step1_total_distance = 0
step1_total_cost = 0
remaining_supply = {i: {r: supply_dict[i][r] for r in resources} for i in supply_nodes}
for i in supply_nodes:
    for j in urgent_nodes:
        for r in resources:
            if model1.x[i, j, r].value and model1.x[i, j, r].value > 0:
                qty = model1.x[i, j, r].value
                distance = qty * distance_dict[(i, j)]
                cost = qty * cost_dict[(i, j, r)]
                step1_total_distance += distance
                step1_total_cost += cost
                remaining_supply[i][r] -= qty
                print(f"Transport {qty:.2f} units of {r} from {i} to {j} (Distance: {distance:.2f}, Cost: {cost:.2f})")

print(f"Total Distance for Step 1: {step1_total_distance:.2f}")
print(f"Total Cost for Step 1: {step1_total_cost:.2f}")

# Step 2: Satisfying the non-Urgent demands

In [ ]:
model2 = pe.ConcreteModel()

model2.x = pe.Var(supply_nodes, demand_nodes, resources, domain=pe.NonNegativeReals)

def obj_rule2(model):
    return sum(
        model2.x[i, j, r] * cost_dict[(i, j, r)]
        for i in supply_nodes
        for j in non_urgent_nodes
        for r in resources
        if (i, j, r) in cost_dict
    )
model2.obj = pe.Objective(rule=obj_rule2, sense=pe.minimize)

def supply_constraint2(model, i, r):
    return sum(model2.x[i, j, r] for j in non_urgent_nodes if (i, j) in distance_dict) <= remaining_supply[i][r]
model2.supply_constraint = pe.Constraint(supply_nodes, resources, rule=supply_constraint2)

def demand_constraint2(model, j, r):
    return sum(model2.x[i, j, r] for i in supply_nodes if (i, j) in distance_dict) >= demand_dict[j][r]
model2.demand_constraint = pe.Constraint(non_urgent_nodes, resources, rule=demand_constraint2)

solver.solve(model2)

print("\nStep 2: Non-Urgent Nodes Results")
step2_total_cost = 0
for i in supply_nodes:
    for j in non_urgent_nodes:
        for r in resources:
            if model2.x[i, j, r].value and model2.x[i, j, r].value > 0:
                qty = model2.x[i, j, r].value
                cost = qty * cost_dict[(i, j, r)]
                step2_total_cost += cost
                print(f"Transport {qty:.2f} units of {r} from {i} to {j} (Cost: {cost:.2f})")

print(f"Total Cost for Step 2: {step2_total_cost:.2f}")

total_cost = step1_total_cost + step2_total_cost
print(f"\nTotal Transportation Cost: {total_cost:.2f}")

# Generating the CSV file including all the transporting plan with cost

In [ ]:
import pandas as pd
import pyomo.environ as pe

distance_data = pd.read_csv('distance_data.csv')
demand_data = pd.read_csv('demand_data.csv')
supply_data = pd.read_csv('supply_data.csv')

supply_nodes = supply_data['supply_node'].unique().tolist()
demand_nodes = demand_data['demand_node'].unique().tolist()
resources = supply_data['resource'].unique().tolist()

supply_dict = supply_data.pivot(index='supply_node', columns='resource', values='capacity').to_dict(orient='index')
demand_dict = demand_data.pivot(index='demand_node', columns='resource', values='demand').to_dict(orient='index')

urgency = demand_data.groupby('demand_node')['urgency'].max().to_dict()
urgent_nodes = [node for node, is_urgent in urgency.items() if is_urgent == 1]
non_urgent_nodes = [node for node, is_urgent in urgency.items() if is_urgent == 0]

distance_dict = {
    (row['supply_node'], row['demand_node']): row['distance']
    for _, row in distance_data.iterrows()
}
cost_dict = {
    (row['supply_node'], row['demand_node'], resource): row[f'{resource}']
    for _, row in distance_data.iterrows()
    for resource in resources
}

output_data = []

model1 = pe.ConcreteModel()

model1.x = pe.Var(supply_nodes, demand_nodes, resources, domain=pe.NonNegativeReals)

def obj_rule1(model):
    return sum(
        model.x[i, j, r] * distance_dict[(i, j)]
        for i in supply_nodes
        for j in urgent_nodes
        for r in resources
        if (i, j) in distance_dict
    )
model1.obj = pe.Objective(rule=obj_rule1, sense=pe.minimize)

def supply_constraint1(model, i, r):
    return sum(model.x[i, j, r] for j in urgent_nodes if (i, j) in distance_dict) <= supply_dict[i][r]
model1.supply_constraint = pe.Constraint(supply_nodes, resources, rule=supply_constraint1)

def demand_constraint1(model, j, r):
    return sum(model.x[i, j, r] for i in supply_nodes if (i, j) in distance_dict) >= demand_dict[j][r]
model1.demand_constraint = pe.Constraint(urgent_nodes, resources, rule=demand_constraint1)

solver = pe.SolverFactory('glpk')
solver.solve(model1)

remaining_supply = {i: {r: supply_dict[i][r] for r in resources} for i in supply_nodes}
for i in supply_nodes:
    for j in urgent_nodes:
        for r in resources:
            if model1.x[i, j, r].value and model1.x[i, j, r].value > 0:
                qty = model1.x[i, j, r].value
                distance = qty * distance_dict[(i, j)]
                cost = qty * cost_dict[(i, j, r)]
                remaining_supply[i][r] -= qty
                output_data.append({
                    "supply_node": i,
                    "demand_node": j,
                    "resource": r,
                    "quantity": qty,
                    "distance": distance,
                    "cost": cost,
                    "step": 1
                })

model2 = pe.ConcreteModel()

model2.x = pe.Var(supply_nodes, demand_nodes, resources, domain=pe.NonNegativeReals)

def obj_rule2(model):
    return sum(
        model2.x[i, j, r] * cost_dict[(i, j, r)]
        for i in supply_nodes
        for j in non_urgent_nodes
        for r in resources
        if (i, j, r) in cost_dict
    )
model2.obj = pe.Objective(rule=obj_rule2, sense=pe.minimize)

def supply_constraint2(model, i, r):
    return sum(model2.x[i, j, r] for j in non_urgent_nodes if (i, j) in distance_dict) <= remaining_supply[i][r]
model2.supply_constraint = pe.Constraint(supply_nodes, resources, rule=supply_constraint2)

def demand_constraint2(model, j, r):
    return sum(model2.x[i, j, r] for i in supply_nodes if (i, j) in distance_dict) >= demand_dict[j][r]
model2.demand_constraint = pe.Constraint(non_urgent_nodes, resources, rule=demand_constraint2)

solver.solve(model2)

for i in supply_nodes:
    for j in non_urgent_nodes:
        for r in resources:
            if model2.x[i, j, r].value and model2.x[i, j, r].value > 0:
                qty = model2.x[i, j, r].value
                cost = qty * cost_dict[(i, j, r)]
                output_data.append({
                    "supply_node": i,
                    "demand_node": j,
                    "resource": r,
                    "quantity": qty,
                    "distance": 0,
                    "cost": cost,
                    "step": 2
                })

output_df = pd.DataFrame(output_data)

output_df.to_csv('transportation_output.csv', index=False)

print("Results saved to 'transportation_output.csv'.")